In [1]:
import pandas as pd
from datetime import date

## Time Slots

In [2]:
# Time Slots
days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri']
n_days = len(days)
times = ['08:00', '08:15', '08:30', '08:45',
         '09:00', '09:15', '09:30', '09:45',
         '10:00', '10:15', '10:30', '10:45',
         '11:00', '11:15', '11:30', '11:45',
         '12:00', '12:15', '12:30', '12:45', '13:00',
         '14:00', '14:15', '14:30', '14:45',
         '15:00', '15:15', '15:30', '15:45',
         '16:00', '16:15', '16:30', '16:45',
         '17:00', '17:15', '17:30', '17:45', '18:00']
n_times = len(times)
lunch_time_index = times.index('13:00')
pre_lunch_times = list(range(lunch_time_index + 1))
post_lunch_times = list(range((lunch_time_index + 1), len(times)))

def begin_end_pairs(session_len:int) -> list:
    pairs = []
    for s in range(0, len(times) - session_len):
        e = s + session_len
        if (((s in pre_lunch_times) and (e in pre_lunch_times)) or
            ((s in post_lunch_times) and (e in post_lunch_times))):
            pairs.append((s,e))
    return pairs
# No lectures start/end in these time slots (as (day, time) tuples) --- labs, tutorials are OK
mon_2pm = (days.index('Mon'), times.index('14:00'),)
wed_2pm = (days.index('Wed'), times.index('14:00'),)
no_lecture_slots = ([(mon_2pm[0], t,) for t in range(mon_2pm[1], (mon_2pm[1] + 6))] +
                    [(wed_2pm[0], t,) for t in range(wed_2pm[1], (wed_2pm[1] + 6))])

def overlap(b:int, e:int, t:int) -> bool:
    # Returns a bool indicating whether the intervel (b,e) contains time t
    # b,e,t are indexes into the 'times' array
    assert (0 <= b < n_times) and(0 <= e < n_times) and (0 <= t < n_times)
    return (b <= t <= e)



## Classrooms

In [3]:
# classroom_cols = {'RoomNum': str, 'Capacity': int, 'Category': str,
#                   'HasProjector': bool, 'HasWacom': bool}
classroom_cols = {'RoomNumber': str, 'Capacity': int}
classrooms = pd.read_csv('classroom_master.csv', dtype=classroom_cols)
n_rooms = classrooms.shape[0]

#get room capacity given room id 
def get_room_capacity(room_id: int) -> int:
    return int(classrooms.loc[room_id, 'Capacity'])

def get_room_category(room_id: int) -> str:
    a=classrooms.loc[room_id, 'Lab']
    b=classrooms.loc[room_id, 'DeptLab']
    if a:
        if b:
            return 'CSE Lab'
        return 'ECE Lab'
    return 'NotLab'

def get_room_num(room_id: int) -> str:
    return classrooms.loc[room_id, 'RoomNumber']

def check_cse_lab_room(room_id: int) -> int:
    if get_room_category(room_id) == 'CSE Lab':
        return True
    return False

def check_ece_lab_room(room_id: int) -> int:
    if get_room_category(room_id) == 'ECE Lab':
        return True
    return False


## Courses & Enrollment

In [4]:
courses = pd.read_csv('course_master.csv')
n_courses = courses.shape[0]
enrollment = pd.read_csv('enrollment_master.csv')
n_enrollments = enrollment.shape[0]
# number of unique emails in enrollment_master_clean.csv
n_students = enrollment['EmailID'].nunique()
students = enrollment['EmailID'].unique()
print(n_students)
course_id = {}
for i in range(n_courses):
    course_id[courses['Code'][i]] = i

course_id_code={}
for i in range(n_courses):
    course_id_code[i] = courses['Code'][i]


def get_course_id(course_code: str) -> int:
    return course_id[course_code]
# make function to get ccourse strength given course id
def get_course_strength(course_id: int) -> int:
    return int(enrollment[enrollment['CourseCode'] == courses['Code'][course_id]].shape[0])
def get_courses_student(student_id: int) -> list:
    l =  enrollment[enrollment['EmailID'] == students[student_id]]['CourseCode'].values
    return [get_course_id(x) for x in l]

def get_course_category(course_id: int) -> str:
    
    a = courses.loc[course_id,'Lab']
    b = courses.loc[course_id,'Dept']
    if a:
        if b == 'CSE':
            return 'CSE Lab'
        return 'ECE Lab'
    return 'NotLab'

def get_not_rooms_course(course_id: int) -> list:
    l=[]
    print("Course:",course_id)
    for i in range(n_rooms):
        if get_room_category(i) != get_course_category(course_id):
            l.append(i)
    print("Rooms:",l)
    return l


# for x in students:
#     print(x)

# for c in range(n_courses):
#     print("Course:",c)
#     print("Course Type:",get_course_category(c))
#     print(get_not_rooms_course(c))



1002


In [5]:

from collections import defaultdict

course_student = defaultdict(list)
intersection_pairs = set()
for i in range(n_students):
    courses_taken = get_courses_student(i)
    for x in courses_taken:
        course_student[x].append(i)
    # add all possible course pairs to intersection_pairs
    for j in range(len(courses_taken)):
        for k in range(j+1, len(courses_taken)):
            intersection_pairs.add((courses_taken[j], courses_taken[k]))
#get unique pairs
intersection_pairs = list(intersection_pairs)
n_intersection_pairs = len(intersection_pairs)
print(intersection_pairs)
print(course_student)

[(55, 57), (7, 17), (26, 30), (7, 26), (7, 35), (48, 54), (21, 37), (6, 57), (10, 27), (41, 42), (10, 36), (25, 43), (3, 6), (44, 56), (3, 24), (14, 24), (14, 33), (14, 51), (15, 16), (47, 48), (17, 44), (18, 21), (29, 36), (48, 49), (21, 32), (6, 43), (29, 45), (2, 27), (25, 29), (2, 36), (2, 45), (22, 32), (13, 51), (3, 28), (16, 56), (55, 56), (7, 16), (5, 46), (5, 55), (17, 57), (6, 29), (29, 31), (6, 38), (6, 56), (33, 37), (10, 35), (10, 44), (25, 42), (51, 54), (14, 32), (5, 41), (28, 43), (6, 24), (9, 57), (29, 35), (29, 44), (6, 42), (21, 40), (39, 48), (20, 53), (2, 26), (13, 32), (43, 45), (24, 41), (35, 41), (32, 54), (13, 50), (16, 46), (16, 55), (28, 29), (5, 27), (9, 25), (5, 36), (47, 51), (5, 45), (6, 10), (17, 56), (29, 30), (6, 46), (2, 3), (10, 25), (24, 27), (32, 40), (3, 4), (24, 36), (35, 36), (35, 45), (24, 45), (13, 54), (9, 29), (6, 14), (6, 23), (42, 44), (39, 47), (2, 7), (12, 39), (20, 52), (12, 48), (4, 44), (23, 57), (13, 22), (24, 31), (43, 44), (13, 49)

In [6]:
print(course_id)

{'AIM 101 ': 0, 'AIM 102 ': 1, 'AIM 704 ': 2, 'AIM 821 ': 3, 'AIM 825 ': 4, 'AIM 829 ': 5, 'AIM 831 ': 6, 'AIM 832 ': 7, 'AIM 843 ': 8, 'AIM 845 ': 9, 'AIM 846 ': 10, 'AMS 401 ': 11, 'AMS 402 ': 12, 'AMS 403 ': 13, 'AMS 404 ': 14, 'COM 605 ': 15, 'COM 837 ': 16, 'COM 863 ': 17, 'CSE 102 ': 18, 'CSE 102 P -A': 19, 'CSE 102 P -B': 20, 'CSE 104 ': 21, 'CSE 212 ': 22, 'CSE 606 ': 23, 'CSE 704 ': 24, 'CSE 718 ': 25, 'CSE 754 ': 26, 'CSE 816 ': 27, 'CSE 819 ': 28, 'CSE 824 ': 29, 'CSE 836 ': 30, 'CSE 852 ': 31, 'DAS 101 ': 32, 'DAS 101P -B': 33, 'DAS 101P -A': 34, 'DAS 605 ': 35, 'DAS 839 ': 36, 'DHS 101B ': 37, 'DHS 108 ': 38, 'DHS 110 ': 39, 'DHS 201 ': 40, 'DHS 305 ': 41, 'DHS 306 ': 42, 'DHS 308 ': 43, 'DHS 314 ': 44, 'DHS 315 ': 45, 'DHS 611 ': 46, 'ECE 212 ': 47, 'ECE 212P -A': 48, 'ECE 303 ': 49, 'ECE 303P -A': 50, 'ECE 304 ': 51, 'EGC 121 ': 52, 'EGC 123 ': 53, 'EGC 301 ': 54, 'VLS 603 ': 55, 'VLS 731 ': 56, 'VLS 804 ': 57}


## Faculty

In [7]:
faculty_cols = {'Name': str, 'Course': str, 'Start': str, 'End': str,
                'PreferredSlot': str, 'Flexibility': int}
faculty = pd.read_csv('faculty_master.csv', dtype=faculty_cols)